In [1]:
import os
import json
import csv
import time
from datetime import datetime, timedelta
from numpy import array
import numpy as np
import pandas as pd


In [2]:
symbols = ["OTO", "HPT", "AMANAH"]
items = os.listdir("INPUT")
items = [i.split('.')[0] for i in items] 

In [24]:
# symbols = ["OTO", "HPT", "AMANAH","BROCK-W1","SPORT-W5","CEN-W4"]
symbols = ["SIMAT-W3", "SIMAT-W2", "AMANAH","BROCK-W1","SPORT-W5","CEN-W4","U","SEAOIL-W"]
# items = ["2017-03-20", "2017-03-21", "2017-03-22"]
items = ["2017-03-15"]

In [4]:
def getFreefloat():
    allShare = 250000000  # ACAP
    freeFloat = 67.03
    return (allShare * freeFloat)/100

def check(list):
    if list[1] is not 0 and list[2] is not 0 and list[4] is 0: return 0
    else: return 1

def getSumVol(list):
    # Sum = list[0] + list[1] + list[2] + list[3] + list[4]
    Sum = np.sum(list)
    return Sum

def event(eve):
    if eve == "B":
        return 1
    elif eve == "S":
        return 0
    else:
        return 5

def getTime(time):
    timeSplit = time.split(':')
    return timeSplit

def getDate(Date):
    dateSplit = Date.split('-')
    return dateSplit


def Timestamp2Datetime(Timestamp):
    Datetime = datetime.fromtimestamp(Timestamp-25200).strftime('%Y-%m-%d-%H-%M-%S')
    # print Datetime
    return Datetime

def Datetime2Timestamp(dt, epoch=datetime(1970,1,1)):
    td = dt - epoch
    # return td.total_seconds()
    return (td.microseconds + (td.seconds + td.days * 86400) * 10**6) // 10**6


def toTemp(price, vol, eve, symbol):
    sumVO = getSumVol(vol)
    compareBid1[symbol] = tempBid[symbol][5]
    compareOff1[symbol] = tempOffer[symbol][5]


    if eve == 1:

        tempBid[symbol] = price + vol

        difBO[symbol] = sumVO - tempBidVol[symbol]

        if difBO[symbol] <= -1000000:
            volBO[symbol] = -4
        elif -1000000 < difBO[symbol] <= -500000:
            volBO[symbol] = -3
        elif -500000 < difBO[symbol] <= -100000:
            volBO[symbol] = -2
        elif -100000 < difBO[symbol] <= -100:
            volBO[symbol] = -1
        elif -100 < difBO[symbol] <= 100000:
            volBO[symbol] = 1
        elif 100000 < difBO[symbol] <= 500000:
            volBO[symbol] = 2
        elif 500000 < difBO[symbol] <= -1000000:
            volBO[symbol] = 3
        elif difBO[symbol] > 1000000:
            volBO[symbol] = 4

        tempBidVol[symbol] = sumVO
        return price + vol + tempOffer[symbol] + [tempBidVol[symbol]] + [tempOffVol[symbol]]
    else:

        tempOffer[symbol] = price + vol

        difBO[symbol] = sumVO - tempOffVol[symbol]

        if difBO[symbol] <= -1000000:
            volBO[symbol] = -40
        elif -1000000 < difBO[symbol] <= -500000:
            volBO[symbol] = -30
        elif -500000 < difBO[symbol] <= -100000:
            volBO[symbol] = -20
        elif -100000 < difBO[symbol] <= -100:
            volBO[symbol] = -10
        elif -100 < difBO[symbol] <= 100000:
            volBO[symbol] = 10
        elif 100000 < difBO[symbol] <= 500000:
            volBO[symbol] = 20
        elif 500000 < difBO[symbol] <= -1000000:
            volBO[symbol] = 30
        elif difBO[symbol] > 1000000:
            volBO[symbol] = 40

        tempOffVol[symbol] = sumVO
        return tempBid[symbol] + price + vol + [tempBidVol[symbol]] + [tempOffVol[symbol]]



def checkTimestamp(timestamp,symbol):
    if timestamp < tempTimestamp[symbol]:
        return tempTimestamp[symbol],1
    else:
        tempTimestamp[symbol] = timestamp
        return timestamp,0










def getSpread(x):
    ''' x<2         0.01    0
        2<=x<5       0.02    1
        5<=x<10      0.05    2
        10<=x<25     0.1     3
        25<=x<100    0.25    4
        100<=x<200   0.5     5
        200<=x<400   1.0     6
        x>=400       2.0     7
    '''
    if x < 200:
        return 0, 1, 200
    elif 200 <= x < 500:
        return 1, .5, 500
    elif 500 <= x < 1000:
        return 2, .2, 1000
    elif 1000 <= x < 2500:
        return 3, .1, 2500
    elif 2500 <= x < 10000:
        return 4, .04, 10000
    elif 10000 <= x < 20000:
        return 5, .02, 20000
    elif 20000 <= x < 40000:
        return 6, .01, 40000
    else:
        return 7, .005, 40000

def getDifSpread(x1, x2):
    a1, t1, z1 = getSpread(x1)
    a2, t2, z2 = getSpread(x2)
    if a1 == a2:
        Dif = (x2 - x1) * t1
    elif a2 > a1:
        Dif = ((x2 - z1) * t2) + ((z1 - x1) * t1)
    else:
        Dif = -(((x1 - z2) * t1) + ((z2 - x2) * t2))
    return int(Dif)



def calmoney(pbid0,pbid1,pbid2,pbid3,pbid4,vbid0,vbid1,vbid2,vbid3,vbid4):
    money = ((pbid0*vbid0)+(pbid1*vbid1)+(pbid2*vbid2)+(pbid3*vbid3)+(pbid4*vbid4))/100
    return money

In [25]:
idSymbol = dict()
tempBid = dict()
tempOffer = dict()
tempBidVol = dict()
tempOffVol = dict()
tempAll = dict()
priATO = dict()
volATO = dict()
tempTimestamp = dict()
marketStatus = dict()
forTemp = dict()
clearNoise = dict()

sumOrder = dict()
countOrder = dict()

lastPrice = dict()
prior = dict()
highPrice = dict()
lowPrice = dict()
avgPrice = dict()
tradeVol = dict()
buyVol = dict()
sellVol = dict()
auctVol = dict()

spread = dict()
count100k = dict()
difBO = dict()

tf30 = dict()
volBO = dict()
typeBO = dict()
compareBid1 = dict()
compareOff1 = dict()


allbidmoney = dict()
sidetransac = dict()



stock =dict()
stock1 =dict()
ato = dict()

tick = dict()

ticklog = dict()
bidtemp = dict()
offertemp = dict()

errorlog = dict()

for symbol in symbols:
    # tempBid[symbol] = [None, None, None, None, None, None, None, None, None, None]
    # tempOffer[symbol] = [None, None, None, None, None, None, None, None, None, None]
    tempBid[symbol] = [0,0,0,0,0,0,0,0,0,0]
    tempOffer[symbol] = [0,0,0,0,0,0,0,0,0,0]
    # forTemp[symbol] = [None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None]

    priATO[symbol] = 0
    volATO[symbol] = 0
    tempTimestamp[symbol] = 0
    marketStatus[symbol] = 0
    clearNoise[symbol] = 0
    idSymbol[symbol] = 0

    sumOrder[symbol] = 0
    countOrder[symbol] = 0

    lastPrice[symbol] = 0
    prior[symbol] = 0
    highPrice[symbol] = 0
    lowPrice[symbol] = 0
    avgPrice[symbol] = 0
    tradeVol[symbol] = 0
    buyVol[symbol] = 0
    sellVol[symbol] = 0
    auctVol[symbol] = 0

    spread[symbol] = 0
    count100k[symbol] = 0
    difBO[symbol] =0
    tf30[symbol] = 0
    volBO[symbol] = 0
    typeBO[symbol] = 0

    tempBidVol[symbol] = 0
    tempOffVol[symbol] = 0
    compareBid1[symbol] = 0
    compareOff1[symbol] = 0

    allbidmoney[symbol] = 0
    sidetransac[symbol] = 0
    
    stock[symbol] = pd.DataFrame()
    stock1[symbol] = pd.DataFrame()
    ato[symbol] = pd.DataFrame()
    
    tick[symbol] = 0
    
    ticklog[symbol] = pd.DataFrame()
    bidtemp[symbol] = []
    offertemp[symbol] = []
    
    errorlog[symbol] = pd.DataFrame()
    

    


for symbol in symbols:
    forTemp[symbol] = []

In [26]:
t0 = time.time()
error = 0

three_tuple = [(current, items[idx - 1] if idx >= 1 else None, items[idx + 1] if idx < len(items) - 1 else None) for idx, current in enumerate(items)]
# return a 3-tuple with current, previous and next elements

for enum, Date in enumerate(items):

    yesterday = three_tuple[enum][1]
    today = three_tuple[enum][0]
    tomorrow = three_tuple[enum][2]
    
    YYYY,MM,DD = getDate(Date)

#     YYYY = int(YYYY)
#     MM = int(MM)
#     DD = int(DD)
    



    with open("INPUT\\" +Date + ".dat") as f:

        content = f.readlines()
        for line in content:
            if 'data' in line:
                try:
                    jsonData = line.replace("data: ", "")
                    jsonDecoded = json.loads(jsonData)
                except ValueError:
                    # print Date, jsonData
                    error += 1
                    continue
                    
                if 'sym' in jsonDecoded.keys():
                    sym = jsonDecoded["sym"]
                    if sym in symbols:
                        if 'isf' in jsonDecoded.keys():
                            isf = jsonDecoded["isf"]
                            
                            priATO[sym] = pri = jsonDecoded["pri"]  # price
                            
                            volATO[sym] = vol = jsonDecoded["vol"]  # volume
                            
                            HH, mm, SS = getTime(jsonDecoded["tim"])
                            HH = int(HH)
                            mm = int(mm)
                            SS = int(SS)
                            
                            if isf == 'F':
                                marketStatus[sym] = 0   #Pre-Open1
                                if HH == 14:
                                    marketStatus[sym] = 2   #Pre-Open2
                                if HH == 16:
                                    marketStatus[sym] = 3  # Pre-Open2
                                    
                            elif isf == 'T':
#                                 marketStatus[sym] = 1
                                if HH == 16 and mm >30:
                                    marketStatus[sym] = 4   #Pre-Close
                            
                        elif 'time' in jsonDecoded.keys():
                            Id = jsonDecoded["id"]
#                             tim = jsonDecoded["time"]
                            tim = checkTimestamp(jsonDecoded["time"],sym)                            
                            
                            sid = jsonDecoded["sid"]
                            pri = jsonDecoded["pri"]
                            vol = jsonDecoded["vol"]
                            
                            
                            if pri[4] == 0:
                                errorlog[sym] = errorlog[sym].append({'error':jsonDecoded}, ignore_index=True)
                                error += 1
                                continue
                            
                            if pri[0] == 0:
                                pri[0] = priATO[sym]
                            
                            

#                             stock[sym] = stock[sym].append(jsonDecoded, ignore_index=True)
                            stock[sym] = stock[sym].append({'btemp':bidtemp[sym] , 'otemp': offertemp[sym], 'marketStatus': marketStatus[sym], 'Id':Id, 'date':Date, 'time':tim[0], 'side':sid, 'p0':pri[0], 'p1':pri[1], 'p2':pri[2], 'p3':pri[3], 'p4':pri[4], 'v0':vol[0], 'v1':vol[1], 'v2':vol[2], 'v3':vol[3], 'v4':vol[4]}, ignore_index=True)
    
                            if sid == 'S':
                                offertemp[sym] = pri + vol
                            elif sid == 'B':
                                bidtemp[sym] = pri + vol
                            else:
                                print jsonData, "in bid offer temp"
                                error += 1
                                continue

#                                 pass
                            
                            
#                             stock[sym] = stock[sym].append({'decode':jsonDecoded,'pri':pri}, ignore_index=True)
#                                 stock[sym] = stock[sym].append({'tf30':tf30[sym], 'Timestamp':Timestamp[0], 'Date':Date}, ignore_index=True)

                        elif 'ava' in jsonDecoded.keys():
                            marketStatus[sym] = 1
            
                            tim = jsonDecoded["tim"]  ##time
                            tim = checkTimestamp(tim,sym)
            
            
            
                            actVol = jsonDecoded["vol"]  ##vol action
                            sid = jsonDecoded["sid"]
                            prc = jsonDecoded["prc"]
                            prior[sym] = prr = jsonDecoded["prr"]  # Previous Close must / 100
                            
                            hgh = jsonDecoded["hgh"]  # high Price must / 100
                            low = jsonDecoded["low"]  # low Price must / 100
                            avg = jsonDecoded["avg"]  # Average Price must / 100
                            
                            avo = jsonDecoded["avo"]
                            bvo = jsonDecoded["bvo"]
                            svo = jsonDecoded["svo"]
                            
                            tick[sym] +=1

                            
                            stock[sym] = stock[sym].append({'btemp':bidtemp[sym] , 'otemp': offertemp[sym], 'marketStatus': marketStatus[sym], 'tick':tick[sym], 'actVol':actVol, 'date':Date, 'time':tim[0], 'side':sid, 'last':prc, 'prior':prr, 'high':hgh, 'low':low, 'average':avg,'tradevol':avo,'buyvol':bvo,'sellvol':svo }, ignore_index=True)
                            
                        else:
                            continue


        for symbol in symbols:
            tempTimestamp[symbol] = 0
            
        t1 = time.time()
        total = t1-t0
        for symbol in symbols:
            tick[symbol] = 0

        print Date, total, "seconds from start time.", " :error = ",error
print "finished."


2017-03-15 173.312999964 seconds from start time.  :error =  363
finished.


In [27]:
dt = dict()

for sym in symbols:
    stock[sym]['Datetime'] = pd.to_datetime(stock[sym]['date'] + ' ' + stock[sym]['time'])
    dt[sym] = stock[sym].set_index(pd.DatetimeIndex(stock[sym]['Datetime']))
    
#     dt[sym] = dt[sym].loc[(dt[sym]['marketStatus'] == 1) & (dt[sym]['p4'] != 0)]
    dt[sym] = dt[sym].loc[(dt[sym]['marketStatus'] == 1)]
    
    

# type(stock['AMANAH']['date'] )


In [28]:
for sym in symbols:
#     stock[sym].to_csv("stock\\" +sym+'ms.csv')
    errorlog[sym].to_csv("stock\\" +sym+'error.csv')
    dt[sym].to_csv("stock\\" +sym+'temp.csv')

In [232]:
dt[sym].loc[(dt[sym]['marketStatus'] == 1) & (dt[sym]['p4'] != 0)]

,Id,date,marketStatus,p0,p1,p2,p3,p4,side,time,...,average,buyvol,high,last,low,prior,sellvol,tick,tradevol,Datetime
2017-02-16 09:57:44,NaN,2017-02-16,1.0,NaN,NaN,NaN,NaN,NaN,U,09:57:44,...,165.0,0.0,165.0,165.0,165.0,165.0,0.0,1.0,114300.0,2017-02-16 09:57:44
2017-02-16 09:57:44,2772,2017-02-16,1.0,165.0,164.0,163.0,162.0,161.0,B,09:57:44,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2017-02-16 09:57:44
2017-02-16 09:57:44,2772,2017-02-16,1.0,167.0,168.0,169.0,170.0,171.0,S,09:57:44,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2017-02-16 09:57:44
2017-02-16 09:57:55,2772,2017-02-16,1.0,165.0,164.0,163.0,162.0,161.0,B,09:57:55,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2017-02-16 09:57:55
2017-02-16 09:57:59,NaN,2017-02-16,1.0,NaN,NaN,NaN,NaN,NaN,S,09:57:59,...,165.0,0.0,165.0,165.0,165.0,165.0,0.0,2.0,115300.0,2017-02-16 09:57:59
2017-02-16 09:57:59,NaN,2017-02-16,1.0,NaN,NaN,NaN,NaN,NaN,S,09:57:59,...,165.0,0.0,165.0,165.0,165.0,165.0,0.0,3.0,125300.0,2017-02-16 09:57:59
2017-02-16 09:57:59,NaN,2017-02-16,1.0,NaN,NaN,NaN,NaN,NaN,S,09:57:59,...,165.0,0.0,165.0,165.0,165.0,165.0,0.0,4.0,160300.0,2017-02-16 09:57:59
2017-02-16 09:57:59,2772,2017-02-16,1.0,165.0,164.0,163.0,162.0,161.0,B,09:57:59,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2017-02-16 09:57:59
2017-02-16 09:58:02,NaN,2017-02-16,1.0,NaN,NaN,NaN,NaN,NaN,B,09:58:02,...,165.0,0.0,167.0,167.0,165.0,165.0,0.0,5.0,165300.0,2017-02-16 09:58:02
2017-02-16 09:58:02,2772,2017-02-16,1.0,167.0,168.0,169.0,170.0,171.0,S,09:58:02,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2017-02-16 09:58:02


In [ ]:
df.groupby(pd.TimeGrouper(freq='30Min'))
